In [1]:
import pandas as pd
import numpy as np
import urllib.request as request
import matplotlib.pyplot as plt
import seaborn as sns
import json
from bs4 import BeautifulSoup
from github import Github
from config import token as token

# pygithub
# include the type of the orgnization and the name of the orgnization
# include all the data, regardless of the size of the repo
# branch cusp winter 2019
# gitignore file to ignore the credentials 

In [2]:
url = 'https://government.github.com/community/'
html_response = request.urlopen(url)
soup = BeautifulSoup(html_response, 'html.parser')

In [3]:
c1, c2, c3, c4, c5 = [], [], [], [], []
for div in soup.find_all('div', id = lambda x: x and x.startswith('include')):
    subDiv = div.find_all('div', id = lambda x: x and x.startswith('type'))
    for child in subDiv:
        leafDiv = child.find_all('div', class_ = lambda x: x and x.endswith('org'))
        for leaf in leafDiv:
            org = leaf.find_all('a')
            for k in org:
                c1.append(div.get('id').split('-')[1])
                c2.append(child.get('id').split('-')[1])
                c3.append(k.get('title'))
                c4.append(k.get('href'))

In [4]:
df = pd.DataFrame()
df['Type'] = c1
df['Country/Division'] = c2
df['Name'] = c3
df['Homepage'] = c4

In [5]:
df.head()

,Type,Country/Division,Name,Homepage
0,governments,argentina,argob,https://github.com/argob
1,governments,argentina,cifasis,https://github.com/cifasis
2,governments,argentina,gcba,https://github.com/gcba
3,governments,argentina,inti-cmnb,https://github.com/inti-cmnb
4,governments,argentina,municipioriogrande,https://github.com/municipioriogrande


In [6]:
# number of repo for each org
# number of contributors
# language of the repo
# l1, l2, l3 = [], [], []
# for org in c3:
#     u = g.get_user(org)
#     rls = u.get_repos()
#     for repo in rls:
        

In [7]:
g = Github(token)
u = g.get_user('argob')
rls = u.get_repos()[0]
rls

Repository(full_name="argob/accesibilidad-web")

In [8]:
rls.language

'HTML'

In [9]:
k = rls.get_collaborators()

In [10]:
k.totalCount # no push access, cannot get number of contributors through pygithub

GithubException: 403 {'message': 'Must have push access to view repository collaborators.', 'documentation_url': 'https://developer.github.com/v3/repos/collaborators/#list-collaborators'}